In [2]:
!pip install langchain langchain-community sentence-transformers chromadb unstructured pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [30]:
!pip install -U langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 2.8 MB/s eta 0:00:00


In [3]:
from google.colab import files
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]

Saving SDV.pdf to SDV.pdf


In [22]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader(pdf_path)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)


In [24]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    encode_kwargs={"normalize_embeddings": True}
)

/tmp/ipython-input-24-1012234954.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [25]:
from langchain_community.vectorstores import Chroma

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory="db"
)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [29]:
import langchain
print(langchain.__version__)


0.3.26


In [ ]:
import os
from langchain_groq import ChatGroq

 
llm = ChatGroq(
    groq_api_key=os.environ["GROQ_API_KEY"],
    model_name="llama-3.3-70b-versatile"  # atau llama3-70b-8192
)


In [47]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

result = qa_chain.invoke("rumusnya apa ya?")
print("Jawaban:", result['result'])
print("\nSumber konten:")
for doc in result['source_documents']:
    print("-", doc.metadata['source'])
    print(doc.page_content[:200], "...\n")


Jawaban: Rumusnya tampaknya terkait dengan Gaussian Copula dan proses Conditional Probability Approximation (CPA). Beberapa rumus yang disebutkan dalam teks adalah:

1. Rumus kovarian: Σ = [[Σuu, Σuk], [Σku, Σkk]]
2. Rumus vektor mean: µ = [µu, µk]
3. Rumus rearrangement matriks Σ dan vektor µ: 
Σ = [[Σuu, Σuk], [Σku, Σkk]]
µ = [µu, µk] = [0, 0]

Namun, tidak jelas apa rumus spesifik yang Anda cari. Jika Anda memiliki pertanyaan lebih lanjut, saya dapat mencoba membantu.

Sumber konten:
- SDV.pdf
Dataset Name Source # Tables # Classes # Exemplars
Biodegradability Relational Repo 5 5 249
Mutagenesis Relational Repo 3 2 145
Airbnb Kaggle 4 12 5000
Rossmann Kaggle 2 8 1017209
Telstra Kaggle 5 3 7 ...

- SDV.pdf
2), (µ1, σ1
2), (µ2, σ2
2)]
Covariance: Σ C=
Distributions: [(µ5, σ5
2), (µ6, σ6
2)]
C55
C65
C56
C66
Fig. 4. An illustration of CPA for a row in table T with primary key
“33”. Tables A, B, and C refer ...

- SDV.pdf
the Σ matrix, as well as the mean vector µ. Initially, µ = 0
due t

In [88]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Custom prompt template
custom_prompt = PromptTemplate.from_template("""
Kamu adalah asisten AI yang ahli membaca dokumen dan menjawab pertanyaan pengguna berdasarkan isi dokumen tersebut.

Berdasarkan konteks berikut, jawablah pertanyaan dengan jelas dan ringkas, dalam bahasa Indonesia yang formal.

### Konteks:
{context}

### Pertanyaan:
{question}

### Jawaban:
""")

# Bangun chain
from langchain.memory import ConversationBufferMemory

chat_history = []

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer",
    k=5
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    output_key="answer",
    combine_docs_chain_kwargs={"prompt": custom_prompt},
    verbose=False
)

In [89]:
response = qa_chain.invoke("Rumus SDV apa ya?")
response["answer"]

'Rumus SDV untuk memperbarui Σ dan µ dengan observasi yang diketahui adalah sebagai berikut:\n\nΣ′ = Σuu - Σuk Σ^-1 kk Σku\nµ′ = µu + Σuk Σ^-1 kk (obs - µk)\n   = Σuk Σ^-1 kk obs\n\nDengan kata lain, rumus ini digunakan untuk memperbarui matriks kovarian (Σ) dan vektor mean (µ) berdasarkan observasi yang diketahui, sehingga dapat digunakan untuk mensintesis data yang realistis.'

In [87]:
from pprint import pprint

pprint(memory.chat_memory.messages)

[HumanMessage(content='Rumus SDV apa ya?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Rumus SDV yang disebutkan dalam konteks tersebut adalah sebagai berikut:\n\nΣ′ = Σuu - ΣukΣ^{-1}kkΣku\nµ′ = µu + ΣukΣ^{-1}kk(obs - µk)\nµ′ = ΣukΣ^{-1}kkobs\n\nRumus ini digunakan untuk memperbarui matriks kovarians (Σ) dan vektor rata-rata (µ) dengan menggunakan observasi yang diketahui untuk mendapatkan nilai baru Σ′ dan µ′ untuk variabel yang tidak diketahui.', additional_kwargs={}, response_metadata={})]


In [97]:
!pip show langchain
!pip show langchain-community
!pip show sentence-transformers
!pip show chromadb
!pip show unstructured
!pip show pypdf
!pip show langchain-groq

Name: langchain
Version: 0.3.26
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
Name: langchain-community
Version: 0.3.27
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Name: sentence-transformers
Version: 4.1.0
Summary: Embeddings, Retrieval, and Reranking
Home-page: https://www.SBERT.net
Author: 
Author-email: Nils Reimers <info@nils-reimers.de>, Tom Aarsen <tom.aarsen@huggingface.co>
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingfa

In [91]:
!python --version

Python 3.11.13


In [100]:
!pip show langchain_huggingface

In [103]:
!pip show absl-py

In [96]:
!pip show keras

Name: keras
Version: 3.8.0
Summary: Multi-backend Keras
Home-page: 
Author: 
Author-email: Keras team <keras-users@googlegroups.com>
License: Apache License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: tensorflow


In [ ]:
pip cache purge